In [1]:
import os, random
from collections import Counter, defaultdict
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr

MAX_LIVES = 6
ALPHABET = [chr(i) for i in range(97, 123)]
ALPHA = 0.5
GAMMA = 0.9
EPSILON = 0.05

# ------------------- Corpus Loader -------------------
def load_corpus_from_file(path):
    if not os.path.exists(path):
        return []
    with open(path, "r", encoding="utf-8") as f:
        words = [w.strip().lower() for w in f if w.strip().isalpha()]
    return [w for w in words if 2 <= len(w) <= 12]

# ------------------- HMM Model -------------------
class HMMModel:
    def __init__(self, corpus):
        self.corpus = corpus
        self.unigram = Counter()
        self.bigram = defaultdict(Counter)
        self.train()

    def train(self):
        for w in self.corpus:
            for ch in w:
                self.unigram[ch] += 1
            for a, b in zip(w, w[1:]):
                self.bigram[a][b] += 1
        total = sum(self.unigram.values()) + 26
        self.unigram_prob = {c: (self.unigram.get(c, 0) + 1) / total for c in ALPHABET}

    def get_probs(self, masked, guessed):
        counts = Counter()
        total = 0
        for w in self.corpus:
            if len(w) != len(masked):
                continue
            if all((masked[i] == "_" or masked[i] == w[i]) for i in range(len(w))):
                if all((g not in w or g in masked) for g in guessed):
                    for i, ch in enumerate(w):
                        if masked[i] == "_":
                            counts[ch] += 1
                            total += 1
        probs = {}
        if total > 0:
            for c in ALPHABET:
                probs[c] = counts[c] / total
        else:
            probs = self.unigram_prob.copy()
        for g in guessed:
            probs[g] = 0
        s = sum(probs.values())
        if s > 0:
            for c in probs:
                probs[c] /= s
        return probs

# ------------------- Hangman Environment -------------------
class HangmanEnv:
    def __init__(self, max_lives=MAX_LIVES):
        self.max_lives = max_lives

    def reset(self, word):
        self.word = word.lower()
        self.masked = ['_' for _ in self.word]
        self.guessed = set()
        self.lives = self.max_lives
        self.done = False
        self.won = False
        self.repeated_guesses = 0
        return ''.join(self.masked)

    def step(self, letter):
        reward = 0
        if letter in self.guessed:
            self.repeated_guesses += 1
            return ''.join(self.masked), -3, False
        self.guessed.add(letter)
        if letter in self.word:
            revealed = 0
            for i, ch in enumerate(self.word):
                if ch == letter and self.masked[i] == '_':
                    self.masked[i] = letter
                    revealed += 1
            reward = 5 + 2 * revealed
        else:
            self.lives -= 1
            reward = -2
        if '_' not in self.masked:
            self.done = True
            self.won = True
            reward += 20
        elif self.lives <= 0:
            self.done = True
            self.won = False
            reward -= 10
        return ''.join(self.masked), reward, self.done

# ------------------- Q-Learning Agent -------------------
class QLearningAgent:
    def __init__(self, hmm):
        self.hmm = hmm
        self.Q = defaultdict(float)

    def get_state(self, masked):
        return ''.join(['L' if ch != '' else '' for ch in masked])

    def select_action(self, masked, guessed):
        state = self.get_state(masked)
        if random.random() < EPSILON:
            options = [c for c in ALPHABET if c not in guessed]
            return random.choice(options)
        probs = self.hmm.get_probs(masked, guessed)
        best, best_val = None, -999
        for c in ALPHABET:
            if c in guessed:
                continue
            q_val = self.Q[(state, c)] + probs[c] * 8
            if q_val > best_val:
                best_val, best = q_val, c
        return best

    def update(self, masked, action, reward, next_masked):
        s, a = self.get_state(masked), action
        ns = self.get_state(next_masked)
        max_next = max([self.Q[(ns, l)] for l in ALPHABET], default=0)
        self.Q[(s, a)] += ALPHA * (reward + GAMMA * max_next - self.Q[(s, a)])

# ------------------- GLOBAL APP -------------------
APP = {"corpus": [], "testset": [], "hmm": None, "agent": None, "env": None}

# ------------------- Upload Training Corpus -------------------
def upload_corpus(file):
    if file is None:
        return "⚠ No file uploaded."
    path = file.name if hasattr(file, "name") else None
    if not path or not os.path.exists(path):
        return "❌ File not found."
    APP["corpus"] = load_corpus_from_file(path)
    APP["hmm"] = HMMModel(APP["corpus"])
    APP["agent"] = QLearningAgent(APP["hmm"])
    APP["env"] = HangmanEnv()
    return f"✅ Loaded training corpus with {len(APP['corpus'])} words."

# ------------------- Upload Test Dataset -------------------
def upload_testset(file):
    if file is None:
        return "⚠ No test file uploaded."
    path = file.name if hasattr(file, "name") else None
    if not path or not os.path.exists(path):
        return "❌ File not found."
    APP["testset"] = load_corpus_from_file(path)
    return f"✅ Loaded test dataset with {len(APP['testset'])} words."

# ------------------- Play Single Word -------------------
def play_word(word):
    if not word or not word.isalpha():
        return "❌ Enter a valid word.", "", ""
    if APP["hmm"] is None:
        return "⚠ Load training corpus first.", "", ""
    env, agent = APP["env"], APP["agent"]
    masked = env.reset(word)
    logs = [f"🎯 Target Word: {word}"]
    for step in range(20):
        if env.done:
            break
        act = agent.select_action(masked, env.guessed)
        next_masked, reward, done = env.step(act)
        agent.update(masked, act, reward, next_masked)
        logs.append(f"Step {step+1}: '{act}' → {next_masked} (r={reward}, lives={env.lives})")
        masked = next_masked
    result = "✅ WON" if env.won else f"❌ LOST ({word})"
    logs.append(result)
    return "\n".join(logs), masked, f"Lives left: {env.lives}"

# ------------------- Evaluate Agent on Test Set (with Accuracy) -------------------
def evaluate_agent(n_games=2000):
    if APP["hmm"] is None or len(APP["testset"]) == 0:
        return "⚠ Load training and test datasets first.", None

    env, agent = APP["env"], APP["agent"]
    test_words = random.sample(APP["testset"], min(n_games, len(APP["testset"])))

    success = wrongs = repeats = 0
    total_correct = total_guesses = 0
    success_trend = []

    for idx, word in enumerate(test_words, 1):
        masked = env.reset(word)
        correct_guesses = 0
        total_guesses_game = 0

        while not env.done:
            act = agent.select_action(masked, env.guessed)
            next_masked, reward, done = env.step(act)
            agent.update(masked, act, reward, next_masked)
            # Count how many new letters were revealed
            if act in word:
                correct_guesses += sum(1 for i, ch in enumerate(word)
                                       if ch == act and masked[i] == "_")
            total_guesses_game += 1
            masked = next_masked

        total_correct += correct_guesses
        total_guesses += total_guesses_game
        if env.won:
            success += 1
        wrongs += (MAX_LIVES - env.lives)
        repeats += env.repeated_guesses
        success_trend.append(success / idx * 100)

    success_rate = (success / len(test_words)) * 100
    avg_wrong = wrongs / len(test_words)
    avg_repeat = repeats / len(test_words)
    accuracy = (total_correct / total_guesses) * 100 if total_guesses > 0 else 0

    # ---- Plot learning curve ----
    plt.figure(figsize=(6, 4))
    plt.plot(success_trend, label="Success Rate (%)", color="blue")
    plt.xlabel("Episodes")
    plt.ylabel("Success Rate (%)")
    plt.title("Agent Learning Curve on Test Set")
    plt.grid(True)
    plt.legend()
    plot_path = "learning_plot.png"
    plt.savefig(plot_path)
    plt.close()

    summary = (
        f"✅ Test Results\n"
        f"Total Games: {len(test_words)}\n"
        f"Success Rate: {success_rate:.2f}%\n"
        f"Accuracy: {accuracy:.2f}%\n"
        f"Avg Wrong Guesses: {avg_wrong:.2f}\n"
        f"Avg Repeated Guesses: {avg_repeat:.2f}"
    )
    return summary, plot_path

# ------------------- Gradio UI -------------------
with gr.Blocks(title="Smart Hangman AI (HMM + Q-Learning + Test Evaluation)") as demo:
    gr.Markdown("## 🧠 Hangman AI — Smarter HMM + Q-Learning + Test Evaluation")

    with gr.Tab("1️⃣ Train & Play"):
        upload_train = gr.File(label="Upload Training Corpus (corpus.txt)")
        train_btn = gr.Button("Load Training Corpus")
        status_train = gr.Textbox(label="Status")
        word_box = gr.Textbox(label="Enter a custom word to test AI manually")
        play_btn = gr.Button("Play Word")
        logs = gr.Textbox(label="Logs", lines=10)
        masked = gr.Textbox(label="Masked Word")
        lives = gr.Textbox(label="Lives")
        train_btn.click(upload_corpus, inputs=[upload_train], outputs=[status_train])
        play_btn.click(play_word, inputs=[word_box], outputs=[logs, masked, lives])

    with gr.Tab("2️⃣ Test & Evaluate"):
        upload_test = gr.File(label="Upload Test Dataset (test.txt)")
        test_btn = gr.Button("Load Test Dataset")
        test_status = gr.Textbox(label="Test Dataset Status")
        num_games = gr.Number(value=2000, label="Number of Test Games (default: 2000)")
        eval_btn = gr.Button("Run Evaluation")
        eval_result = gr.Textbox(label="Evaluation Results", lines=6)
        eval_plot = gr.Image(label="Learning Curve")
        test_btn.click(upload_testset, inputs=[upload_test], outputs=[test_status])
        eval_btn.click(evaluate_agent, inputs=[num_games], outputs=[eval_result, eval_plot])

print("🚀 Launching Smart Hangman AI with Test Evaluation...")
demo.launch(share=True)

🚀 Launching Smart Hangman AI with Test Evaluation...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://41c086fd603c82a870.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
